In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [6]:
from pyspark.sql import SparkSession

# Spark oturumu oluşturun
spark = SparkSession.builder \
    .appName("HDFSToCSV") \
    .getOrCreate()

# HDFS üzerindeki CSV dosyasının yolunu belirtin
hdfs_file_path = "/ProjectTweets.csv"

# CSV dosyasını Spark DataFrame'e okuyun
df = spark.read.csv(hdfs_file_path, header=False, inferSchema=True)

In [7]:

# DataFrame'i görüntüleyin veya işlem yapın
df.show(5)


+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
+---+----------+--------------------+--------+---------------+--------------------+
only showing top 5 rows



In [2]:
from pyspark.sql import SparkSession

# Spark oturumu oluşturun
spark = SparkSession.builder \
    .appName("DFToMongoDB") \
    .getOrCreate()

# Veri çerçevenizi oluşturun veya mevcut bir veri çerçevesini kullanın
data = [("John", 28), ("Alice", 22), ("Bob", 32)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)


In [3]:
df.show(1)

+----+---+
|Name|Age|
+----+---+
|John| 28|
+----+---+
only showing top 1 row



In [7]:
df.write.format("mongodb") \
    .option("uri","mongodb://127.0.0.1:27001/") \
    .option("database","sample_db") \
    .option("collection","scb") \
    .mode("append").save()

In [3]:
from pyspark.sql import SparkSession

# Spark oturumu oluşturun
spark = SparkSession.builder \
    .appName("DFToMySQL") \
    .getOrCreate()

# Veri çerçevenizi oluşturun veya mevcut bir veri çerçevesini kullanın
data = [("John", 28), ("Alice", 22), ("Bob", 32)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

# MySQL veritabanı bağlantısı yapılandırması
mysql_options = {
    "url": "jdbc:mysql://localhost:3306/sample",  # MySQL bağlantı URL'si
    "driver": "com.mysql.cj.jdbc.Driver",  # MySQL JDBC sürücüsü
    "dbtable": "yourtable",  # Hedef MySQL tablo adı
    "user": "root",  # MySQL kullanıcı adı
    "password": "password"  # MySQL parola
}

# DataFrame'i MySQL veritabanına yükleyin
df.write.format("jdbc").options(**mysql_options).mode("overwrite").save()


In [5]:
!pwd

/home/hduser/Desktop


In [6]:
!cat zahid.txt

zahid
